    000_16_230114_141_subway_transer_28
    
    data :028_우이신설_주소데이터_20221122.csv
     save : subway_info_28.csv
     display : subway_info_28.html

In [61]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [62]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [63]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [64]:
#path = 'c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/001_수도권1호선_주소데이터_20221122.csv'

subway_info_df = pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/028_우이신설_주소데이터_20221122.csv',encoding='cp949')
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,우이신설경전철주식회사,우이신설,4.19민주묘지,서울특별시 강북구 우이동 72-186,서울특별시 강북구 삼양로 519,NaN,NaN,NaN,NaN,NaN
1,우이신설경전철주식회사,우이신설,가오리,서울특별시 강북구 수유동 338-46,서울특별시 강북구 삼양로 426,NaN,NaN,NaN,NaN,NaN
2,우이신설경전철주식회사,우이신설,보문,서울특별시 성북구 보문동1가 127-6,서울특별시 성북구 보문로 116,NaN,NaN,NaN,NaN,NaN
3,우이신설경전철주식회사,우이신설,북한산보국문,서울특별시 성북구 정릉동 286-19,서울특별시 성북구 보국문로 88,NaN,NaN,NaN,NaN,NaN
4,우이신설경전철주식회사,우이신설,북한산우이,서울특별시 강북구 우이동 16-20,서울특별시 강북구 삼양로 676,NaN,NaN,NaN,NaN,NaN


In [65]:
subway_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   철도운영기관명     13 non-null     object 
 1   선명          13 non-null     object 
 2   역명          13 non-null     object 
 3   지번주소        13 non-null     object 
 4   도로명주소       13 non-null     object 
 5   Unnamed: 5  0 non-null      float64
 6   Unnamed: 6  0 non-null      float64
 7   Unnamed: 7  0 non-null      float64
 8   Unnamed: 8  0 non-null      float64
 9   Unnamed: 9  0 non-null      float64
dtypes: float64(5), object(5)
memory usage: 1.1+ KB


In [66]:
# subway_info_df['주소'] = subway_info_df['주소'].astype(str)

In [67]:
#5  university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [68]:
#5  university .. 주소 기반 위도,경도 추가
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_adress_x_y(row["지번주소"])
     
        subway_info_df.loc[_, "x"] = x
        subway_info_df.loc[_, "y"] = y
    except:
        print(row["지번주소"] + '은(는) 검색에 실패했습니다')

subway_info_df.head()


,철도운영기관명,선명,역명,지번주소,도로명주소,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,x,y
0,우이신설경전철주식회사,우이신설,4.19민주묘지,서울특별시 강북구 우이동 72-186,서울특별시 강북구 삼양로 519,NaN,NaN,NaN,NaN,NaN,957079.5214505,1961280.20147171
1,우이신설경전철주식회사,우이신설,가오리,서울특별시 강북구 수유동 338-46,서울특별시 강북구 삼양로 426,NaN,NaN,NaN,NaN,NaN,957359.557499309,1960395.60677326
2,우이신설경전철주식회사,우이신설,보문,서울특별시 성북구 보문동1가 127-6,서울특별시 성북구 보문로 116,NaN,NaN,NaN,NaN,NaN,957602.495536728,1954049.77325465
3,우이신설경전철주식회사,우이신설,북한산보국문,서울특별시 성북구 정릉동 286-19,서울특별시 성북구 보국문로 88,NaN,NaN,NaN,NaN,NaN,956638.564435424,1957007.25845593
4,우이신설경전철주식회사,우이신설,북한산우이,서울특별시 강북구 우이동 16-20,서울특별시 강북구 삼양로 676,NaN,NaN,NaN,NaN,NaN,957041.214456384,1962652.45715937


In [69]:
#5  university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [70]:
#5  university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        subway_info_df.loc[_, "lng"] = x
        subway_info_df.loc[_, "lat"] = y

    except:
        pass
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,x,y,lng,lat
0,우이신설경전철주식회사,우이신설,4.19민주묘지,서울특별시 강북구 우이동 72-186,서울특별시 강북구 삼양로 519,NaN,NaN,NaN,NaN,NaN,957079.5214505,1961280.20147171,127.013446,37.650009
1,우이신설경전철주식회사,우이신설,가오리,서울특별시 강북구 수유동 338-46,서울특별시 강북구 삼양로 426,NaN,NaN,NaN,NaN,NaN,957359.557499309,1960395.60677326,127.016672,37.642049
2,우이신설경전철주식회사,우이신설,보문,서울특별시 성북구 보문동1가 127-6,서울특별시 성북구 보문로 116,NaN,NaN,NaN,NaN,NaN,957602.495536728,1954049.77325465,127.019794,37.584864
3,우이신설경전철주식회사,우이신설,북한산보국문,서울특별시 성북구 정릉동 286-19,서울특별시 성북구 보국문로 88,NaN,NaN,NaN,NaN,NaN,956638.564435424,1957007.25845593,127.008701,37.611475
4,우이신설경전철주식회사,우이신설,북한산우이,서울특별시 강북구 우이동 16-20,서울특별시 강북구 삼양로 676,NaN,NaN,NaN,NaN,NaN,957041.214456384,1962652.45715937,127.012931,37.662375


In [71]:
#5  university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [72]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in subway_info_df.iterrows():
    try:
        folium.Marker(
            location=[subway_info_df.loc[idx, "lat"], subway_info_df.loc[idx, "lng"]],
            popup=subway_info_df.loc[idx, "역명"],
            icon=folium.Icon(color="cadetblue", icon="flag")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [73]:
#5 seoul university 
map

In [74]:
map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway_info_28.html")


In [75]:
#apt_df.to_excel("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/apt_cn_sung.xls", index = False)
# subway_info_df.to_csv("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_info_02.csv", index = False)
subway_info_df.to_csv("subway_info_28.csv", index = False)
